In [2]:
import pandas as pd
import numpy as np


## Abrir o arquivo CSV

In [3]:
df = pd.read_csv(r'C:\Users\berna\Teste_Analytics_Bernardo_Timm\data\raw\vendas_raw.csv')

## Algumas Informações Relevantes

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514 entries, 0 to 513
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          514 non-null    int64  
 1   Data        514 non-null    object 
 2   Produto     514 non-null    object 
 3   Categoria   514 non-null    object 
 4   Quantidade  501 non-null    float64
 5   Preco       514 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 24.2+ KB


In [5]:
df.head()

,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,2023-05-17,Carregador USB,Acessórios,8.0,0.00
1,2,2023-05-21,Mouse Gamer,Acessórios,8.0,136.34
2,3,2023-11-14,Fone Bluetooth,Acessórios,6.0,139.82
3,4,2024-01-01,iPhone,Telefonia,10.0,4671.46
4,5,2023-09-19,Notebook HP,Informática,6.0,4145.99


In [6]:
df.describe()

,ID,Quantidade,Preco
count,514.000000,501.000000,514.000000
mean,250.145914,8.169661,1648.068405
std,145.570479,4.159947,1851.164898
min,1.000000,1.000000,0.000000
25%,123.250000,5.000000,159.045000
50%,250.500000,8.000000,223.465000
75%,376.750000,12.000000,3481.455000
max,500.000000,15.000000,5749.740000


In [7]:
df.isnull().sum()

ID             0
Data           0
Produto        0
Categoria      0
Quantidade    13
Preco          0
dtype: int64

# Tratamento de valores nulos

In [8]:
# Utilizando o valos médio do produto como método para suprir os valores nulos
df['Quantidade'] = df.groupby('Produto')['Quantidade'].transform(lambda x: x.fillna(x.median()))

In [9]:
df.isnull().sum()

ID            0
Data          0
Produto       0
Categoria     0
Quantidade    0
Preco         0
dtype: int64

# Convertendo Tipos de Dados

In [10]:
# Convertendo a coluna de data
df['Data'] = pd.to_datetime(df['Data'])

# Garantindo que Quantidade seja inteiro
df['Quantidade'] = df['Quantidade'].astype(int)

# Garantindo que Preço seja float
df['Preco'] = df['Preco'].astype(float)

In [11]:
df.dtypes

ID                     int64
Data          datetime64[ns]
Produto               object
Categoria             object
Quantidade             int32
Preco                float64
dtype: object

# Remoção de Duplicados

In [12]:
# Número de registros antes de excluir duplicados
len(df)

514

In [13]:
df=df.drop_duplicates()

In [14]:
# Número após
len(df)

500

# Verificação de dados inválidos

In [15]:
# Verificando datas fora do período
datas_invalidas = df[~df['Data'].between('2023-01-01', '2023-12-31')]

len(datas_invalidas)

17

In [16]:
# Verificando preços negativos
precos_invalidos = df[df['Preco'] <= 0]

len(precos_invalidos)

9

In [17]:
# Verificando quantidades negativas
qtd_invalidas = df[df['Quantidade'] <= 0]

len(qtd_invalidas)

0

In [18]:
# Removendo registros inválidos
df = df[df['Data'].between('2023-01-01', '2023-12-31')]

# Removendo registros com preços ou quantidades inválidas
df = df[df['Preco'] > 0]
df = df[df['Quantidade'] > 0]

len(df)

474

# Correção de categorias 

In [19]:
# Dicionário com as categorias corretas 
categorias_corretas = {
    'Notebook Dell': 'Informática',
    'Notebook HP': 'Informática',
    'Mouse Gamer': 'Acessórios',
    'Teclado Wireless': 'Acessórios',
    'Monitor 24"': 'Informática',
    'Smartphone Samsung': 'Telefonia',
    'iPhone': 'Telefonia',
    'Carregador USB': 'Acessórios',
    'Webcam HD': 'Acessórios',
    'Fone Bluetooth': 'Acessórios'
}


In [20]:
# Verificando se há produtos com categoria diferente das apresentadas no dicionário
for index, row in df.iterrows():
    if row['Categoria'] != categorias_corretas[row['Produto']]:
        print(f"Produto: {row['Produto']}")
        print(f"Categoria atual: {row['Categoria']}")
        print(f"Categoria correta: {categorias_corretas[row['Produto']]}")
len(row['Categoria'])


Produto: Teclado Wireless
Categoria atual: Telefonia
Categoria correta: Acessórios
Produto: Webcam HD
Categoria atual: Telefonia
Categoria correta: Acessórios
Produto: Webcam HD
Categoria atual: Telefonia
Categoria correta: Acessórios
Produto: Notebook Dell
Categoria atual: Acessórios
Categoria correta: Informática
Produto: Mouse Gamer
Categoria atual: Telefonia
Categoria correta: Acessórios
Produto: Webcam HD
Categoria atual: Telefonia
Categoria correta: Acessórios
Produto: Monitor 24"
Categoria atual: Telefonia
Categoria correta: Informática
Produto: Notebook HP
Categoria atual: Telefonia
Categoria correta: Informática
Produto: Smartphone Samsung
Categoria atual: Informática
Categoria correta: Telefonia
Produto: iPhone
Categoria atual: Informática
Categoria correta: Telefonia
Produto: Notebook Dell
Categoria atual: Telefonia
Categoria correta: Informática
Produto: Teclado Wireless
Categoria atual: Telefonia
Categoria correta: Acessórios
Produto: iPhone
Categoria atual: Acessórios
Cat

11

In [21]:
# Corrigindo as categorias utilizando a função map
df['Categoria'] = df['Produto'].map(categorias_corretas)

In [22]:
# Verificando o resultado 
print(df.groupby('Categoria')['Produto'].count())

Categoria
Acessórios     244
Informática    142
Telefonia       88
Name: Produto, dtype: int64


# Calculando o Total de vendas

In [ ]:
#Criando uma nova coluna
df['Valor_Total'] = df['Quantidade'] * df['Preco']

In [31]:
produto_mais_vendido = df.groupby('Produto')['Quantidade'].sum().sort_values(ascending=False)
produto_mais_vendido

Produto
Teclado Wireless      548
Notebook HP           481
Fone Bluetooth        412
Webcam HD             412
Smartphone Samsung    401
Mouse Gamer           362
Notebook Dell         335
iPhone                330
Monitor 24"           329
Carregador USB        261
Name: Quantidade, dtype: int32

In [33]:
df.sample(50)

,ID,Data,Produto,Categoria,Quantidade,Preco,Valor_Total
312,313,2023-12-06,Notebook Dell,Informática,8,4361.02,34888.16
445,446,2023-06-07,iPhone,Telefonia,13,5098.95,66286.35
10,11,2023-01-20,iPhone,Telefonia,12,4410.25,52923.00
48,49,2023-02-11,"Monitor 24""",Informática,1,1082.78,1082.78
355,356,2023-12-14,Smartphone Samsung,Telefonia,3,2256.62,6769.86
479,480,2023-10-01,Teclado Wireless,Acessórios,15,227.09,3406.35
166,167,2023-10-07,Smartphone Samsung,Telefonia,8,2438.23,19505.84
163,164,2023-06-15,"Monitor 24""",Informática,7,1288.19,9017.33
146,147,2023-12-14,Carregador USB,Acessórios,14,53.52,749.28
72,73,2023-06-02,Webcam HD,Acessórios,7,155.64,1089.48


# Salvando o arquivo

In [ ]:
df.to_csv('../data/processed/data_clean.csv', index=False)